# Demonstration of `pdfplumber`'s `extract_table` method.

This notebook uses our [example PDF](../pdfs/background-checks.pdf) from the FBI's National Instant Criminal Background Check System.

### Import `pdfplumber`

In [1]:
import pdfplumber

### Load the PDF

In [2]:
pdf = pdfplumber.from_path("../pdfs/background-checks.pdf")

### Get the first page

In [3]:
page_1 = pdf.pages[0]

### Use `.crop` to focus on the main data table

It starts around 80px from the top, and is about 405px tall. To select only these characters, we use `within_bbox`, and pass a bounding box of `(0, 80, PDF_WIDTH, 485)` as the `(x0, top, x1, bottom)` values.

In [4]:
table_crop = page_1.crop((0, 80, page_1.width, 485))

### Use `extract_table` to pull the data

- Because the columns are separated by lines, we use `v="lines"`
- Because the rows are, primarily, separated by gutters, we use `h="gutters"`
- And because side-by-side characters don't abut one another exactly, we use `x_tolerance=2`.

In [5]:
table = table_crop.extract_table(v="lines", h="gutters", x_tolerance=2)
for row in table[:5]:
    print(row)

['Alabama', '18,870', '23,022', '22,650', '859', '1,178', '0', '14', '15', '0', '2,179', '2,307', '11', '0', '0', '0', None, None, '13', '14', '0', '3', '2', '0', '71,137']
['Alaska', '209', '3,062', '3,209', '191', '184', '0', '9', '3', '0', '100', '100', '0', '18', '9', '1', None, None, '0', '0', '0', '0', '0', '0', '7,095']
['Arizona', '2,303', '12,382', '9,041', '707', '618', '0', '5', '3', '0', '1,273', '648', '4', '76', '8', '1', None, None, '9', '6', '1', '1', '1', '0', '27,087']
['Arkansas', '3,298', '6,359', '11,611', '168', '376', '0', '12', '6', '1', '922', '2,275', '1', '0', '0', '0', None, None, '6', '12', '1', '0', '0', '0', '25,048']
['California', '98452', '41181', '35007', '4559', '0', '0', '0', '0', '0', '480', '433', '4', '0', '0', '0', None, None, '0', '0', '0', '0', '0', '0', '180116']


### Convert keys and values to something more useful

The dictionary keys returned by `extract_columns` are simply numbered in order, `0, 1, 2, ...`. Let's add the actual column names, and also convert strings-representing-numbers to the numbers themselves, e.g., `"18,870" -> 18870`:

In [6]:
COLUMNS = [
    "state",
    "permit",
    "handgun",
    "long_gun",
    "other",
    "multiple",
    "admin",
    "prepawn_handgun",
    "prepawn_long_gun",
    "prepawn_other",
    "redemption_handgun",
    "redemption_long_gun",
    "redemption_other",
    "returned_handgun",
    "returned_long_gun",
    "returned_other",
    "rentals_handgun",
    "rentals_long_gun",
    "private_sale_handgun",
    "private_sale_long_gun",
    "private_sale_other",
    "return_to_seller_handgun",
    "return_to_seller_long_gun",
    "return_to_seller_other",
    "totals"
]

In [7]:
def parse_value(i, x):
    if i == 0: return x
    if x == None: return None
    return int(x.replace(",", ""))

In [8]:
def parse_row(row):
    return dict((COLUMNS[i], parse_value(i, cell))
        for i, cell in enumerate(row))

In [9]:
parsed_table = [ parse_row(row) for row in table ]

Here's the first row, parsed:

In [10]:
parsed_table[0]

{'admin': 0,
 'handgun': 23022,
 'long_gun': 22650,
 'multiple': 1178,
 'other': 859,
 'permit': 18870,
 'prepawn_handgun': 14,
 'prepawn_long_gun': 15,
 'prepawn_other': 0,
 'private_sale_handgun': 13,
 'private_sale_long_gun': 14,
 'private_sale_other': 0,
 'redemption_handgun': 2179,
 'redemption_long_gun': 2307,
 'redemption_other': 11,
 'rentals_handgun': None,
 'rentals_long_gun': None,
 'return_to_seller_handgun': 3,
 'return_to_seller_long_gun': 2,
 'return_to_seller_other': 0,
 'returned_handgun': 0,
 'returned_long_gun': 0,
 'returned_other': 0,
 'state': 'Alabama',
 'totals': 71137}

### Sort the data

For demonstration purposes, let's list the rows with the highest number of handgun-only background checks:

In [11]:
for row in list(reversed(sorted(parsed_table, key=lambda x: x["handgun"])))[:6]:
    print("{state}: {handgun:,d} handgun-only checks".format(**row))

Totals: 671,330 handgun-only checks
Pennsylvania: 62,752 handgun-only checks
Texas: 56,941 handgun-only checks
Florida: 50,796 handgun-only checks
California: 41,181 handgun-only checks
Ohio: 34,878 handgun-only checks


### Use `collate_chars` to extract the report month

It looks like the month of the report is listed in an area 35px to 65px from the top of the page. But there's also some other text directly above and below it. So when we crop for that area, we'll use `strict=True` to select only characters (and other objects) that are fully within the crop-box.

In [12]:
month_crop = page_1.crop((0, 35, page_1.width, 65), strict=True)

In [13]:
month_chars = month_crop.collate_chars(x_tolerance=2, y_tolerance=2)
month_chars

'November - 2015'

---

---

---